In [190]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [27]:
# read in our data
happy = pd.read_csv("/Users/Philip/Schools/TAMU/STAT_636/homework/happiness.csv")
# keep only the columns we care about
happy = happy[happy.columns[range(4,10+1)]]
happy.head()


,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [160]:

X_train = happy[happy.columns[range(0,6)]]
# generate intercept column - this will have the same length as the other column
one_col = pd.DataFrame(np.ones(X_train.shape[0]))
one_col.rename(columns={0:'int'}, inplace=True  )

# merge these
X_train = pd.concat([one_col, X_train], axis=1)

# response
Y_train = pd.DataFrame(happy[happy.columns[6]])

In [183]:
# create a loop to execute LOOCV

def loocv_lm(X_train, Y_train):

    a = range(X_train.shape[0])
    mse_list = []

    for i in range(X_train.shape[0]):
        # design matrix
        loo_dat = X_train.iloc[a[:i] + a[(i+1):], :]
        valid_dat = X_train.iloc[a[i], :]
        valid_dat = valid_dat.values.reshape(1, -1) # manipulation required for sklearn predictions

        # response vector
        loo_resp = Y_train.iloc[a[:i] + a[(i+1):], :]
        valid_resp = Y_train.iloc[a[i], :]

        loo_regr = linear_model.LinearRegression(fit_intercept=False) # we have already handled this
        loo_regr.fit(loo_dat, loo_resp)

        # make a prediction
        loo_pred = loo_regr.predict(valid_dat)

        # calculate the mean-squared error
        mse = mean_squared_error(valid_resp, loo_pred)
        mse_list.append(mse)

    # get the final mse for all of the models    
    mse_tot = sum(mse_list) / len(mse_list)
    
    return mse_tot, mse_list

In [191]:
mse_tot, mse_list = loocv_lm(X_train=X_train, Y_train=Y_train)

In [207]:
# bootstrapping function for our list of squared errors

def loo_bootstrap(x, B=10000):
    """
    function to simulate a sampling distribution for our iterative MSE values.
    x needs to be in list format
    """
    # holding shell
    mean_list = []
    
    for i in range(0, B):
        hold_samp = np.random.choice(x, size=len(x), replace=True)
        hold_mean = hold_samp.mean()
        mean_list.append(hold_mean)
    
    return mean_list


In [214]:
boot = loo_bootstrap(x=mse_list)
# compare
print(sum(boot) / len(boot))
print(mse_tot)

0.307270497821
0.306910236066


0.30732361116574614